## Data cleaning 

**Import Modules**

In [96]:
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA, OneHotEncoder, StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import isnan, when, count, col

In [97]:
import os

# Data Manipulation
import numpy as np 
import pandas as pd

In [98]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

**Read csv file**

In [99]:
filename = 'data.csv'
df = spark.read.csv(filename, inferSchema=True, header = True).limit(100)

In [100]:
type(df)

pyspark.sql.dataframe.DataFrame

**Converting**

In [101]:
floats = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "loudness",
         "speechiness", "tempo", "valence"]
for i in floats:
        df = df.withColumn(i, df[i].cast(FloatType()))

ints = ["duration_ms", "explicit", "key", "mode", "popularity", "year"]
for i in ints:
    df = df.withColumn(i, df[i].cast(IntegerType()))

In [102]:
# Check Process: whether it works
df

DataFrame[acousticness: float, artists: string, danceability: float, duration_ms: int, energy: float, explicit: int, id: string, instrumentalness: float, key: int, liveness: float, loudness: float, mode: int, name: string, popularity: int, release_date: string, speechiness: float, tempo: float, valence: float, year: int]

In [103]:
# After converting, create a New Dataset called df.
df.createOrReplaceTempView("df")
spark.sql("select * from df").show(5)

+------------+--------------------+------------+-----------+------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-------+-------+----+
|acousticness|             artists|danceability|duration_ms|energy|explicit|                  id|instrumentalness|key|liveness|loudness|mode|                name|popularity|release_date|speechiness|  tempo|valence|year|
+------------+--------------------+------------+-----------+------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-------+-------+----+
|       0.991|     ['Mamie Smith']|       0.598|     168333| 0.224|       0|0cS0A1fUEUd1EW3Fc...|         5.22E-4|  5|   0.379| -12.628|   0|Keep A Song In Yo...|        12|        1920|     0.0936|149.976|  0.634|1920|
|       0.643|"[""Screamin' Jay...|       0.852|     150200| 0.517|       0|0hbkKFIJm7Z05H8Zl...|          0.0264|  5|  

In [104]:
df.printSchema()

root
 |-- acousticness: float (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: float (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- energy: float (nullable = true)
 |-- explicit: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: float (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- release_date: string (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- year: integer (nullable = true)



**Drop Useless Variables & Reduce Duplicates**

In [105]:
# Cleaning
# Variables id, name, release_date are not related to our question.
# Thus, droping all three.
df_drop = df.drop("id", "name", "release_date")

In [106]:
# Artists is the only remaining categorical feature
df_drop.printSchema()

root
 |-- acousticness: float (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: float (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- energy: float (nullable = true)
 |-- explicit: integer (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: float (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- year: integer (nullable = true)



In [107]:
# Count rows and unique rows
print('Rows = {}'.format(df_drop.count()))
print('Distinct Rows = {}'.format(df_drop.distinct().count()))

# Check
# Drop all duplicates 
df1 = df_drop.dropDuplicates()
print('The number of rows with duplicate data removed = {}'.format(df1.count()))

Rows = 100
Distinct Rows = 98
The number of rows with duplicate data removed = 98


**Check and Delete Null**

In [108]:
df1.select([count(when(isnan(c), c)).alias(c) for c in df1.columns]).show()

+------------+-------+------------+-----------+------+--------+----------------+---+--------+--------+----+----------+-----------+-----+-------+----+
|acousticness|artists|danceability|duration_ms|energy|explicit|instrumentalness|key|liveness|loudness|mode|popularity|speechiness|tempo|valence|year|
+------------+-------+------------+-----------+------+--------+----------------+---+--------+--------+----+----------+-----------+-----+-------+----+
|           0|      0|           0|          0|     0|       0|               0|  0|       0|       0|   0|         0|          0|    0|      0|   0|
+------------+-------+------------+-----------+------+--------+----------------+---+--------+--------+----+----------+-----------+-----+-------+----+



In [109]:
df1.na.drop().show(1)

+------------+---------------+------------+-----------+------+--------+----------------+---+--------+--------+----+----------+-----------+-------+-------+----+
|acousticness|        artists|danceability|duration_ms|energy|explicit|instrumentalness|key|liveness|loudness|mode|popularity|speechiness|  tempo|valence|year|
+------------+---------------+------------+-----------+------+--------+----------------+---+--------+--------+----+----------+-----------+-------+-------+----+
|       0.991|['Mamie Smith']|       0.598|     168333| 0.224|       0|         5.22E-4|  5|   0.379| -12.628|   0|        12|     0.0936|149.976|  0.634|1920|
+------------+---------------+------------+-----------+------+--------+----------------+---+--------+--------+----+----------+-----------+-------+-------+----+
only showing top 1 row



**Numeric Variables & Categorical Variables**

In [110]:
num_cols = []
cat_cols = []

for s in df1.schema:
    data_type = str(s.dataType)
    if data_type == "StringType":
        cat_cols.append(s.name)
    
    #if data_type == "FloatType" or data_type == "IntType" or data_type =="String":
    else:
        num_cols.append(s.name)

In [111]:
num_cols

['acousticness',
 'danceability',
 'duration_ms',
 'energy',
 'explicit',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'popularity',
 'speechiness',
 'tempo',
 'valence',
 'year']

In [112]:
cat_cols

['artists']

In [113]:
df1.describe().toPandas()

,summary,acousticness,artists,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
0,count,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98
1,mean,0.6006128165351806,None,0.5824744904375806,231907.24489795917,0.4449275505767033,0.14285714285714285,0.31306140813942374,5.142857142857143,0.21727959252893925,-12.392306128326727,0.6530612244897959,0.7653061224489796,0.09290408196726016,113.26907122865015,0.5380499978378719,1920.0
2,stddev,0.41954522589353777,None,0.1749440912956772,100351.28096245402,0.22824666237275687,0.35172622905632944,0.3907475863752577,3.560956665158118,0.14870300774369233,5.304858143688915,0.47844336976134183,2.5758634641097196,0.11392275857497776,27.316508375543872,0.24590656279759582,0.0
3,min,1.16E-5,"""[""""Screamin' Jay Hawkins""""]""",0.0735,77477,0.0148,0,0.0,0,0.0494,-33.187,0,0,0.0253,59.544,0.0224,1920
4,max,0.996,['Ánna Khrisáfi'],0.936,774360,0.998,1,0.983,11,0.805,-4.107,1,17,0.845,183.958,0.95,1920


## Linear Regression Model

**SCALING**

In [114]:
from pyspark.sql.functions import col
import pyspark.sql.functions as func
df1 = df1.withColumn('popularity_final',
                     func.round(df1['popularity']/100, 2))

**SELECT AND STANDARDIZE FEATURES**

In [115]:
vars_to_keep = ['popularity_final','acousticness', 'danceability', 'duration_ms', 'energy', 
                'explicit', 'instrumentalness', 'key', 'liveness', 'loudness', 
                'mode', 'speechiness', 'tempo', 'valence', 'year']

# subset the dataframe on these predictors
df2 = df1.select(vars_to_keep)
df2.show(1)

+----------------+------------+------------+-----------+------+--------+----------------+---+--------+--------+----+-----------+-------+-------+----+
|popularity_final|acousticness|danceability|duration_ms|energy|explicit|instrumentalness|key|liveness|loudness|mode|speechiness|  tempo|valence|year|
+----------------+------------+------------+-----------+------+--------+----------------+---+--------+--------+----+-----------+-------+-------+----+
|            0.12|       0.991|       0.598|     168333| 0.224|       0|         5.22E-4|  5|   0.379| -12.628|   0|     0.0936|149.976|  0.634|1920|
+----------------+------------+------------+-----------+------+--------+----------------+---+--------+--------+----+-----------+-------+-------+----+
only showing top 1 row



In [116]:
from pyspark.ml.linalg import DenseVector
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

In [117]:
feature_cols = ['acousticness','danceability', 'duration_ms', 'energy', 
                'explicit', 'instrumentalness', 'key', 'liveness', 'loudness', 
                'mode', 'speechiness', 'tempo', 'valence', 'year']

assembler = VectorAssembler(
    inputCols = feature_cols, 
    outputCol = "features") 
df2 = assembler.transform(df2)

In [118]:
df2 = df2.select(["popularity_final",'features']) \
         .withColumnRenamed("popularity_final", 'label')
df2.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
| 0.12|[0.99099999666213...|
| 0.07|[0.64300000667572...|
| 0.04|[0.99299997091293...|
| 0.17|[1.72999993083067...|
| 0.02|[0.29499998688697...|
+-----+--------------------+
only showing top 5 rows



In [119]:
## Feature scaling
# Initialize the `standardScaler`
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled", 
                                withStd=True, withMean=False)

# Fit the DataFrame to the scaler; this computes the mean, standard deviation of each feature
scaler = standardScaler.fit(df2)

# Transform the data in `df2` with the scaler
scaled_df2 = scaler.transform(df2)

In [120]:
scaled_df2.show(5)

+-----+--------------------+--------------------+
|label|            features|     features_scaled|
+-----+--------------------+--------------------+
| 0.12|[0.99099999666213...|[2.36208145272427...|
| 0.07|[0.64300000667572...|[1.53261190210488...|
| 0.04|[0.99299997091293...|[2.36684845787022...|
| 0.17|[1.72999993083067...|[4.12351237496781...|
| 0.02|[0.29499998688697...|[0.70314228045068...|
+-----+--------------------+--------------------+
only showing top 5 rows



In [121]:
# Split data into train set (80%), test set (20%) 
splits = scaled_df2.randomSplit([0.8, 0.2])
train_df = splits[0]
test_df = splits[1]

In [122]:
train_df.show(5)

+-----+--------------------+--------------------+
|label|            features|     features_scaled|
+-----+--------------------+--------------------+
|  0.0|[1.75000004674075...|[4.17118331644375...|
|  0.0|[1.02999998489394...|[2.45503922181518...|
|  0.0|[1.80999995791353...|[4.31419510032234...|
|  0.0|[0.00478000007569...|[0.01139328916332...|
|  0.0|[0.00675999978557...|[0.01611268432664...|
+-----+--------------------+--------------------+
only showing top 5 rows



In [123]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='label', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Intercept: 0.005866666666666667


In [124]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 0.019602
r2: 0.000000


In [29]:
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

import pyspark.sql.functions as func

In [125]:
feature_cols = ['acousticness', 'danceability', 'duration_ms', 'energy', 
                'explicit', 'instrumentalness', 'key', 'liveness', 'loudness', 
                'mode', 'speechiness', 'tempo', 'valence', 'year']

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features", handleInvalid="skip")

# Scaling feature columns using Standard Scaler

scaler = StandardScaler(inputCol="features", outputCol="features_scaled") 

# Reducing number of features to 5 using PCA

pca = PCA(inputCol="features_scaled", outputCol="features_pca", k=5)

# Initializing a Linear Regression Model

regression = LinearRegression(featuresCol="features_pca", labelCol="popularity")

# Creating Pipeline model to run all the stages together

pipeline = Pipeline(stages = [assembler, scaler, pca, regression])

# Lets train the model with 80% od the data

train, test = df.randomSplit([0.8, 0.2])

model = pipeline.fit(train)

# Lets get predictions from the model created

predictions = model.transform(test)

# Let us check the accuracy of the model

evaluator = RegressionEvaluator() \
                 .setPredictionCol("prediction") \
                 .setLabelCol("popularity") \
                 .setMetricName("rmse")

print("RMSE error value: ", evaluator.evaluate(predictions))

RMSE error value:  1.2605050107201896
